## Regression-2: ridge vs ols by holdout and cross-validation

最小２乗回帰とリッジ回帰モデルを構築しモデル性能とその中身を比較してみましょう。<br>データはボストン・ハウジングデータを使います。

In [ ]:
# import data for regression
import pandas as pd
from IPython.core.display import display
from sklearn.datasets import load_boston

# set data by role
dataset = load_boston()
X = pd.DataFrame(dataset.data,
                 columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

# check the shape
print('--------------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('--------------------------------------------')
display(X.join(y).head())

OLSとRidgeのどちらが良い予測モデルかをholdoutにより検証してみましょう。このデータでは、OLSとリッジ回帰に大きな性能差は見られないと思います。ただし、リッジ回帰のalphaを大きくすると、係数総和が減少していく様子が確認できます。

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# ホールドアウトのためデータを訓練とテストに分割
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.20,
                                                 random_state=1)
# make pipelines
pipelines = {
   'ols': Pipeline([('scl',StandardScaler()), ('est',LinearRegression())])
  ,'ridge1': Pipeline([('scl',StandardScaler()),('est',Ridge(alpha=1.0))])
  ,'ridge2': Pipeline([('scl',StandardScaler()),('est',Ridge(alpha=20.0))])
}

# build models
scores = {}
for pipe_name, est in pipelines.items():
    est.fit(X_train, y_train)
    scores[('train',pipe_name)]=r2_score(y_train, est.predict(X_train))
    scores[('test',pipe_name)]=r2_score(y_test, est.predict(X_test))

display(pd.Series(scores).unstack())
                                        
# 回帰係数の総和比較
# リッジ回帰の正則化項の役割把握（モデルの「性能」評価ではない）
print('OLS coefficient total:%.6f'%(np.absolute(pipelines['ols'].named_steps['est'].coef_).sum()))
print('Ridge coefficient total:%.6f'%(np.absolute(pipelines['ridge1'].named_steps['est'].coef_).sum()))
print('Ridge coefficient total:%.6f'%(np.absolute(pipelines['ridge2'].named_steps['est'].coef_).sum()))

次に、交差検証（k-fold法）によって評価してみます。holdoutでは見られなかった評価値のブレに気付くでしょう。評価値がブレる場合、(1)学習データに存在する外れ値の処理の見直し、(2)よりシンプルなアルゴリズムの検討、(2)データサンプル数の拡充が基本的な打ち手となりますが、ここでは、<b>交差検証によって、holdoutでは見逃していた可能性のあるモデル精度の堅牢性（robustness）に対する懸念を把握し得ること</b>を理解しましょう。

In [ ]:
from sklearn.model_selection import cross_val_score

# build models
scores={}
for pipe_name, est in pipelines.items():
    cv_results = cross_val_score(est,
                                 X,
                                 y,
                                 cv=5,
                                 scoring='r2')
    print('----------')
    print('algorithm:', pipe_name)
    print('cv_results:', cv_results)
    print('avg +- std_dev', cv_results.mean(),'+-', cv_results.std())

<b>[確認してみよう]</b>hodloutによる確認時のrandom_stateを1から0に変更してみよう。評価値のブレは観測されるだろうか？